In [1]:
from dataclasses import dataclass
from typing import List

import numpy as np


@dataclass
class UnitCommitmentData:
    demand: float
    pmin: List[float]
    pmax: List[float]
    cfix: List[float]
    cvar: List[float]

In [2]:
import gurobipy as gp
from gurobipy import GRB, quicksum
from typing import Union
from miplearn.io import read_pkl_gz
from miplearn.solvers.gurobi import GurobiModel


def build_uc_model(data: Union[str, UnitCommitmentData]) -> GurobiModel:
    if isinstance(data, str):
        data = read_pkl_gz(data)

    model = gp.Model()
    n = len(data.pmin)
    x = model._x = model.addVars(n, vtype=GRB.BINARY, name="x")
    y = model._y = model.addVars(n, name="y")
    model.setObjective(
        quicksum(data.cfix[i] * x[i] + data.cvar[i] * y[i] for i in range(n))
    )
    model.addConstrs(y[i] <= data.pmax[i] * x[i] for i in range(n))
    model.addConstrs(y[i] >= data.pmin[i] * x[i] for i in range(n))
    model.addConstr(quicksum(y[i] for i in range(n)) == data.demand)
    return GurobiModel(model)

In [3]:
model = build_uc_model(
    UnitCommitmentData(
        demand=100.0,
        pmin=[10, 20, 30],
        pmax=[50, 60, 70],
        cfix=[700, 600, 500],
        cvar=[1.5, 2.0, 2.5],
    )
)

model.optimize()
print("obj =", model.inner.objVal)
print("x =", [model.inner._x[i].x for i in range(3)])
print("y =", [model.inner._y[i].x for i in range(3)])

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 7 rows, 6 columns and 15 nonzeros
Model fingerprint: 0x58dfdd53
Variable types: 3 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+01]
  Objective range  [2e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 6 rows and 3 columns
Presolve time: 0.01s
Presolved: 1 rows, 3 columns, 3 nonzeros
Variable types: 0 continuous, 3 integer (1 binary)
Found heuristic solution: objective 1990.0000000

Root relaxation: objective 1.320000e+03, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestB

In [4]:
from scipy.stats import uniform
from typing import List
import random


def random_uc_data(samples: int, n: int, seed: int = 42) -> List[UnitCommitmentData]:
    random.seed(seed)
    np.random.seed(seed)
    pmin = uniform(loc=100_000.0, scale=400_000.0).rvs(n)
    pmax = pmin * uniform(loc=2.0, scale=2.5).rvs(n)
    cfix = pmin * uniform(loc=100.0, scale=25.0).rvs(n)
    cvar = uniform(loc=1.25, scale=0.25).rvs(n)
    return [
        UnitCommitmentData(
            demand=pmax.sum() * uniform(loc=0.5, scale=0.25).rvs(),
            pmin=pmin,
            pmax=pmax,
            cfix=cfix,
            cvar=cvar,
        )
        for _ in range(samples)
    ]

In [5]:
from miplearn.io import write_pkl_gz

data = random_uc_data(samples=500, n=500)
train_data = write_pkl_gz(data[0:450], "uc/train")
test_data = write_pkl_gz(data[450:500], "uc/test")

In [6]:
from miplearn.collectors.basic import BasicCollector

bc = BasicCollector()
bc.collect(train_data, build_uc_model)

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from miplearn.components.primal.actions import SetWarmStart
from miplearn.components.primal.mem import (
    MemorizingPrimalComponent,
    MergeTopSolutions,
)
from miplearn.extractors.fields import H5FieldsExtractor

comp = MemorizingPrimalComponent(
    clf=KNeighborsClassifier(n_neighbors=25),
    extractor=H5FieldsExtractor(
        instance_fields=["static_constr_rhs"],
    ),
    constructor=MergeTopSolutions(25, [0.0, 1.0]),
    action=SetWarmStart(),
)

In [8]:
from miplearn.solvers.learning import LearningSolver

solver_ml = LearningSolver(components=[comp], )
solver_ml.fit(train_data)
solver_ml.optimize(test_data[0], build_uc_model)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1001 rows, 1000 columns and 2500 nonzeros
Model fingerprint: 0xa8b70287
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+08, 3e+08]
Presolve removed 1000 rows and 500 columns
Presolve time: 0.01s
Presolved: 1 rows, 500 columns, 500 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.6166537e+09   5.648803e+04   0.000000e+00      0s
       1    8.2906219e+09   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  8.290621916e+09

User-callback calls 56, time in user-callback 0.00 sec
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0

/usr/local/share/c3/v8/conda/envs/new_pso/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


{'WS: Count': 1, 'WS: Number of variables set': 480.0}

In [9]:
solver_baseline = LearningSolver(components=[])
solver_baseline.fit(train_data)
solver_baseline.optimize(test_data[0], build_uc_model);

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1001 rows, 1000 columns and 2500 nonzeros
Model fingerprint: 0xa8b70287
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+08, 3e+08]
Presolve removed 1000 rows and 500 columns
Presolve time: 0.01s
Presolved: 1 rows, 500 columns, 500 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.6166537e+09   5.648803e+04   0.000000e+00      0s
       1    8.2906219e+09   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.290621916e+09

User-callback calls 56, time in user-callback 0.00 sec
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0